##### IMPORTING RELEVANT PACKAGES AND LIBRARIES

In [1]:
import numpy as np
import os
import SimpleITK as sitk
from sklearn.model_selection import train_test_split
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers import Input, Concatenate, Conv3D, Flatten, Dense
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.losses import BinaryCrossentropy
from sklearn.metrics import confusion_matrix, accuracy_score, roc_curve, auc
import matplotlib.pyplot as plt

##### CONVERTING 2D SLICES TO 3D VOLUME

In [2]:
# Define the root directory where all participant folders are located
root_directory = r"C:\Users\Lenovo\Downloads\PROSTATE CANCER FILES\PROSTATEx"

# Function to convert a list of DICOM files into a 3D image
def convert_to_3D(dicom_files, output_path):
    reader = sitk.ImageSeriesReader()
    reader.SetFileNames(dicom_files)
    image = reader.Execute()
    sitk.WriteImage(image, output_path)

# Iterate through each participant folder
for participant_folder in os.listdir(root_directory):
    participant_path = os.path.join(root_directory, participant_folder)
    
    # Check if it's a directory
    if os.path.isdir(participant_path):
        # Iterate through each subfolder (t2, adc, dwi)
        for subfolder in os.listdir(participant_path):
            subfolder_path = os.path.join(participant_path, subfolder)
            
            # Check if it's a directory
            if os.path.isdir(subfolder_path):
                dicom_files = []
                # Collect all DICOM files in the subfolder
                for root, dirs, files in os.walk(subfolder_path):
                    for file in files:
                        if file.endswith('.dcm'):
                            filename = os.path.join(root, file)
                            dicom_files.append(filename)
                
                # Define the output path for the 3D image
                # Adjust the output filename based on the subfolder name
                if "t2tsetra" in subfolder.lower():
                    output_filename = 't2.dcm'
                elif "ep2ddifftraDYNDIST" in subfolder.lower() or "ep2ddifftraDYNDISTMIX" in subfolder.lower():
                    output_filename = 'dwi.dcm'
                elif "ep2ddifftraDYNDISTADC" in subfolder.lower() or "ep2ddifftraDYNDISTMIXADC" in subfolder.lower():
                    output_filename = 'adc.dcm'
                else:
                    output_filename = '3D_image.dcm' # Default name if no specific condition met
                
                output_path = os.path.join(subfolder_path, output_filename)
                
                # Convert the 2D images to a 3D image
                convert_to_3D(dicom_files, output_path)


##### COREGISTRATION OF T2 AND ADC IMAGES

In [3]:
# Define the root directory where all participant folders are located
root_directory = r"C:\Users\Lenovo\Downloads\PROSTATE CANCER FILES\PROSTATEx"

def register_images(fixed_image_path, moving_image_path, output_path):
    fixed_image = sitk.ReadImage(fixed_image_path, sitk.sitkFloat32)
    moving_image = sitk.ReadImage(moving_image_path, sitk.sitkFloat32)
    
    print(fixed_image.GetSize())
    print(moving_image.GetSize())

    print(fixed_image.GetSpacing())
    print(moving_image.GetSpacing())
    
    initial_transform = sitk.CenteredTransformInitializer(sitk.Cast(fixed_image, moving_image.GetPixelID()), 
                                                          moving_image, 
                                                          sitk.Euler3DTransform(), 
                                                          sitk.CenteredTransformInitializerFilter.GEOMETRY)
    
    registration_method = sitk.ImageRegistrationMethod()
    registration_method.SetMetricAsMattesMutualInformation(numberOfHistogramBins=50)
    registration_method.SetMetricSamplingStrategy(registration_method.RANDOM)
    registration_method.SetMetricSamplingPercentage(0.01)
    registration_method.SetInterpolator(sitk.sitkLinear) 
    registration_method.SetOptimizerAsGradientDescent(learningRate=1, numberOfIterations=100, convergenceMinimumValue=1e-6)
    registration_method.SetOptimizerScalesFromPhysicalShift() 
    registration_method.SetInitialTransform(initial_transform, inPlace=False)
    
    final_transform = registration_method.Execute(sitk.Cast(fixed_image, sitk.sitkFloat32), 
                                                 sitk.Cast(moving_image, sitk.sitkFloat32))
    
    moving_resampled = sitk.Resample(moving_image, fixed_image, final_transform, sitk.sitkLinear, 0.0)
    
    print(moving_resampled.GetSize())
    print(moving_resampled.GetSpacing())
    
    sitk.WriteImage(sitk.Cast(moving_resampled, sitk.sitkUInt16), output_path)

# Iterate through each participant folder
for participant_folder in os.listdir(root_directory):
    participant_path = os.path.join(root_directory, participant_folder)
    
    # Check if it's a directory
    if os.path.isdir(participant_path):
        # Find all T2 and ADC images in their respective subfolders
        t2_images = []
        adc_images = []
        for subdir, dirs, files in os.walk(participant_path):
            for file in files:
                if "t2" in file.lower() and file.endswith('.dcm'):
                    t2_images.append(os.path.join(subdir, file))
                elif "adc" in file.lower() and file.endswith('.dcm'):
                    adc_images.append(os.path.join(subdir, file))
        
        # Register each T2 image with each ADC image
        for i, (t2_image, adc_image) in enumerate(zip(t2_images, adc_images)):
            output_path = os.path.join(participant_path, f"adccor_{i}.dcm")
            register_images(t2_image, adc_image, output_path)

(384, 384, 19)
(84, 128, 19)
(0.5, 0.5, 2.9999999941936846)
(2.0, 2.0, 2.9999999941936846)
(384, 384, 19)
(0.5, 0.5, 2.9999999941936846)
(384, 384, 19)
(84, 128, 19)
(0.5, 0.5, 2.9999999749107253)
(2.0, 2.0, 2.99999997491072)
(384, 384, 19)
(0.5, 0.5, 2.9999999749107253)
(384, 384, 19)
(84, 128, 19)
(0.5, 0.5, 2.9999999684424354)
(2.0, 2.0, 2.999999968442332)
(384, 384, 19)
(0.5, 0.5, 2.9999999684424354)
(384, 384, 21)
(84, 128, 19)
(0.5, 0.5, 2.9999999861695534)
(2.0, 2.0, 3.0000000797893085)
(384, 384, 21)
(0.5, 0.5, 2.9999999861695534)
(384, 384, 19)
(84, 128, 19)
(0.5, 0.5, 4.500000116956479)
(2.0, 2.0, 4.500000116956532)
(384, 384, 19)
(0.5, 0.5, 4.500000116956479)
(384, 384, 19)
(84, 128, 19)
(0.5, 0.5, 2.9999999971342004)
(2.0, 2.0, 2.9999999971342004)
(384, 384, 19)
(0.5, 0.5, 2.9999999971342004)
(384, 384, 19)
(84, 128, 19)
(0.5, 0.5, 2.9999999974902654)
(2.0, 2.0, 2.9999999974902036)
(384, 384, 19)
(0.5, 0.5, 2.9999999974902654)
(384, 384, 23)
(84, 128, 19)
(0.5, 0.5, 2.99999

##### COREGISTRATION OF T2 AND DWI IMAGES

In [4]:
# Define the root directory where all participant folders are located
root_directory = r"C:\Users\Lenovo\Downloads\PROSTATE CANCER FILES\PROSTATEx"

def register_images(fixed_image_path, moving_image_path, output_path):
    fixed_image = sitk.ReadImage(fixed_image_path, sitk.sitkFloat32)
    moving_image = sitk.ReadImage(moving_image_path, sitk.sitkFloat32)
    
    print(fixed_image.GetSize())
    print(moving_image.GetSize())

    print(fixed_image.GetSpacing())
    print(moving_image.GetSpacing())
    
    initial_transform = sitk.CenteredTransformInitializer(sitk.Cast(fixed_image, moving_image.GetPixelID()), 
                                                          moving_image, 
                                                          sitk.Euler3DTransform(), 
                                                          sitk.CenteredTransformInitializerFilter.GEOMETRY)
    
    registration_method = sitk.ImageRegistrationMethod()
    registration_method.SetMetricAsMattesMutualInformation(numberOfHistogramBins=50)
    registration_method.SetMetricSamplingStrategy(registration_method.RANDOM)
    registration_method.SetMetricSamplingPercentage(0.01)
    registration_method.SetInterpolator(sitk.sitkLinear) 
    registration_method.SetOptimizerAsGradientDescent(learningRate=1, numberOfIterations=100, convergenceMinimumValue=1e-6)
    registration_method.SetOptimizerScalesFromPhysicalShift() 
    registration_method.SetInitialTransform(initial_transform, inPlace=False)
    
    final_transform = registration_method.Execute(sitk.Cast(fixed_image, sitk.sitkFloat32), 
                                                 sitk.Cast(moving_image, sitk.sitkFloat32))
    
    moving_resampled = sitk.Resample(moving_image, fixed_image, final_transform, sitk.sitkLinear, 0.0)
    
    print(moving_resampled.GetSize())
    print(moving_resampled.GetSpacing())
    
    sitk.WriteImage(sitk.Cast(moving_resampled, sitk.sitkUInt16), output_path)

# Iterate through each participant folder
for participant_folder in os.listdir(root_directory):
    participant_path = os.path.join(root_directory, participant_folder)
    
    # Check if it's a directory
    if os.path.isdir(participant_path):
        # Find all T2 and DWI images in their respective subfolders
        t2_images = []
        dwi_images = []
        for subdir, dirs, files in os.walk(participant_path):
            for file in files:
                if "t2" in file.lower() and file.endswith('.dcm'):
                    t2_images.append(os.path.join(subdir, file))
                elif "dwi" in file.lower() and file.endswith('.dcm'):
                    dwi_images.append(os.path.join(subdir, file))
        
        # Register each T2 image with each DWI image
        for i, (t2_image, dwi_image) in enumerate(zip(t2_images, dwi_images)):
            output_path = os.path.join(participant_path, f"dwicor_{i}.dcm")
            register_images(t2_image, dwi_image, output_path)

(384, 384, 19)
(84, 128, 57)
(0.5, 0.5, 2.9999999941936846)
(2.0, 2.0, 0.9642857124193985)
(384, 384, 19)
(0.5, 0.5, 2.9999999941936846)
(384, 384, 19)
(84, 128, 57)
(0.5, 0.5, 2.9999999749107253)
(2.0, 2.0, 0.9642857062213028)
(384, 384, 19)
(0.5, 0.5, 2.9999999749107253)
(384, 384, 19)
(84, 128, 57)
(0.5, 0.5, 2.9999999684424354)
(2.0, 2.0, 0.9642857041421781)
(384, 384, 19)
(0.5, 0.5, 2.9999999684424354)
(384, 384, 21)
(84, 128, 57)
(0.5, 0.5, 2.9999999861695534)
(2.0, 2.0, 0.9642857399322777)
(384, 384, 21)
(0.5, 0.5, 2.9999999861695534)
(384, 384, 19)
(84, 128, 57)
(0.5, 0.5, 4.500000116956479)
(2.0, 2.0, 1.4464286090217422)
(384, 384, 19)
(0.5, 0.5, 4.500000116956479)
(384, 384, 19)
(84, 128, 57)
(0.5, 0.5, 2.9999999971342004)
(2.0, 2.0, 0.9642857133645644)
(384, 384, 19)
(0.5, 0.5, 2.9999999971342004)
(384, 384, 19)
(84, 128, 57)
(0.5, 0.5, 2.9999999974902654)
(2.0, 2.0, 0.964285713478994)
(384, 384, 19)
(0.5, 0.5, 2.9999999974902654)
(384, 384, 23)
(84, 128, 57)
(0.5, 0.5, 2.99

##### CROPPING REGION OF INTEREST

In [1]:
import os
import SimpleITK as sitk

# Define the root directory where all participant folders are located
root_directory = r"C:\Users\Lenovo\Downloads\PROSTATE CANCER FILES\PROSTATEx"

# Define the size of the cropped region
crop_size = (180, 180, 19)

# Iterate through each participant folder
for participant_folder in os.listdir(root_directory):
    participant_path = os.path.join(root_directory, participant_folder)
    
    # Check if it's a directory
    if os.path.isdir(participant_path):
        # Iterate through each image in the subfolder
        for image_name in ["t2.dcm", "adccor_0.dcm", "dwicor_0.dcm"]:
            # Load the DICOM image
            input_path = os.path.join(participant_path, image_name)
            input_image = sitk.ReadImage(input_path)

            # Define the center coordinate for cropping (based on the almost lesion coordinate)
            center_coordinate = (178, 140, 0)

            # Calculate the starting index for cropping
            start_index = [int(center_coordinate[i] - crop_size[i] / 2) for i in range(3)]

            # Ensure the cropping region lies within the bounds of the input image
            for i in range(3):
                if start_index[i] < 0:
                    start_index[i] = 0
                elif start_index[i] + crop_size[i] > input_image.GetSize()[i]:
                    start_index[i] = input_image.GetSize()[i] - crop_size[i]

            # Define the region to crop
            region_to_crop = [slice(start_index[i], start_index[i] + crop_size[i]) for i in range(3)]

            # Crop the image
            cropped_image = input_image[region_to_crop]

            # Save the cropped image
            output_path = os.path.join(participant_path, f"{image_name[:-4]}_cropped.dcm")
            sitk.WriteImage(cropped_image, output_path)

            # Print the shape of the cropped image
            print(f"Shape of the cropped {image_name[:-4]} image:", cropped_image.GetSize())


Shape of the cropped t2 image: (180, 180, 19)
Shape of the cropped adccor_0 image: (180, 180, 19)
Shape of the cropped dwicor_0 image: (180, 180, 19)
Shape of the cropped t2 image: (180, 180, 19)
Shape of the cropped adccor_0 image: (180, 180, 19)
Shape of the cropped dwicor_0 image: (180, 180, 19)
Shape of the cropped t2 image: (180, 180, 19)
Shape of the cropped adccor_0 image: (180, 180, 19)
Shape of the cropped dwicor_0 image: (180, 180, 19)
Shape of the cropped t2 image: (180, 180, 19)
Shape of the cropped adccor_0 image: (180, 180, 19)
Shape of the cropped dwicor_0 image: (180, 180, 19)
Shape of the cropped t2 image: (180, 180, 19)
Shape of the cropped adccor_0 image: (180, 180, 19)
Shape of the cropped dwicor_0 image: (180, 180, 19)
Shape of the cropped t2 image: (180, 180, 19)
Shape of the cropped adccor_0 image: (180, 180, 19)
Shape of the cropped dwicor_0 image: (180, 180, 19)
Shape of the cropped t2 image: (180, 180, 19)
Shape of the cropped adccor_0 image: (180, 180, 19)
Sh

##### PERFORMING STANDARDIZATION